<a href="https://colab.research.google.com/github/zelalemgetahun9374/Swahili-Speech-To-Text/blob/main/swahilli_speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import os
import pickle
import librosa
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from IPython.display import Audio

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import * 
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
os.chdir("/content/drive/MyDrive/structured_data/train/wav")

## Inputs

In [86]:
rows = []
parent_dir = "./"
files = os.listdir(parent_dir)
for f in files[:1100]:
    audio, fs = librosa.load(f"{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows.append(row)
rows[:5]

[{'audio': array([-0.01944764, -0.02380558, -0.01945851, ..., -0.02546581,
         -0.01958567, -0.00813534], dtype=float32),
  'filename': 'SWH-15-20101109_emission_swahili_15h00_-_16h00_tu_20101109_part198'},
 {'audio': array([ 6.4895018e-03, -9.7726457e-05, -6.6772741e-03, ...,
          4.5560386e-02,  3.6010724e-02,  0.0000000e+00], dtype=float32),
  'filename': 'SWH-15-20101109_emission_swahili_15h00_-_16h00_tu_20101109_part411'},
 {'audio': array([-0.01290035, -0.01468785, -0.01265284, ..., -0.16431093,
         -0.11925378, -0.05920244], dtype=float32),
  'filename': 'SWH-15-20101109_emission_swahili_15h00_-_16h00_tu_20101109_part23'},
 {'audio': array([-0.0225578 , -0.02763871, -0.01984122, ..., -0.08003497,
         -0.08758974,  0.        ], dtype=float32),
  'filename': 'SWH-15-20101109_emission_swahili_15h00_-_16h00_tu_20101109_part233'},
 {'audio': array([0.00551369, 0.00701467, 0.00613299, ..., 0.07596262, 0.06502344,
         0.        ], dtype=float32),
  'filename': 

In [85]:
len(files)

10180

In [87]:
sample_audios = []
for row in rows:
    audio = row['audio']
    sample_audios.append(audio)
sample_audios[:5]

[array([-0.01944764, -0.02380558, -0.01945851, ..., -0.02546581,
        -0.01958567, -0.00813534], dtype=float32),
 array([ 6.4895018e-03, -9.7726457e-05, -6.6772741e-03, ...,
         4.5560386e-02,  3.6010724e-02,  0.0000000e+00], dtype=float32),
 array([-0.01290035, -0.01468785, -0.01265284, ..., -0.16431093,
        -0.11925378, -0.05920244], dtype=float32),
 array([-0.0225578 , -0.02763871, -0.01984122, ..., -0.08003497,
        -0.08758974,  0.        ], dtype=float32),
 array([0.00551369, 0.00701467, 0.00613299, ..., 0.07596262, 0.06502344,
        0.        ], dtype=float32)]

In [88]:
meta_df = pd.read_csv('/content/drive/MyDrive/metadata.csv')

In [89]:
meta_df.head()

,filename,transcription,filepath,sample_rate,duration
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.14
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.10
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.65
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.90
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,2.94


In [90]:
txts = []
for row in rows:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts.append(txt)

txts[:5]

[array([['janga la ukimwi unajua sana sana tunavyojua lilitokea uganda sehemu za rakai']],
       dtype=object),
 array([['hali ya mambo baada tu ya kupata mada hii kwamba']], dtype=object),
 array([['mamlaka za afya nchini haiti zinathibitisha kuwa kesi mia moja na ishirini zimeripotiwa']],
       dtype=object),
 array([['watu wafuga lakini pigo']], dtype=object),
 array([['kwanza kutolewa kwa matokeo ya awali lakini hadi sasa']],
       dtype=object)]

In [91]:
txts = np.array(txts).reshape(-1)

In [92]:
txts[:5]

array(['janga la ukimwi unajua sana sana tunavyojua lilitokea uganda sehemu za rakai',
       'hali ya mambo baada tu ya kupata mada hii kwamba',
       'mamlaka za afya nchini haiti zinathibitisha kuwa kesi mia moja na ishirini zimeripotiwa',
       'watu wafuga lakini pigo',
       'kwanza kutolewa kwa matokeo ya awali lakini hadi sasa'],
      dtype=object)

In [93]:
clean_txts = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts.append(clean_txt)

In [94]:
clean_txts[:5]

['janga la ukimwi unajua sana sana tunavyojua lilitokea uganda sehemu za rakai',
 'hali ya mambo baada tu ya kupata mada hii kwamba',
 'mamlaka za afya nchini haiti zinathibitisha kuwa kesi mia moja na ishirini zimeripotiwa',
 'watu wafuga lakini pigo',
 'kwanza kutolewa kwa matokeo ya awali lakini hadi sasa']

In [95]:
'' in clean_txts

True

In [96]:
df = pd.DataFrame(clean_txts)
df.columns = ['texts']
df.head()

,texts
0,janga la ukimwi unajua sana sana tunavyojua li...
1,hali ya mambo baada tu ya kupata mada hii kwamba
2,mamlaka za afya nchini haiti zinathibitisha ku...
3,watu wafuga lakini pigo
4,kwanza kutolewa kwa matokeo ya awali lakini ha...


In [97]:
idxs = df[df['texts'] == ''].index
idxs

Int64Index([43, 158, 240], dtype='int64')

In [98]:
del clean_txts[idxs[-1]]
del clean_txts[idxs[-2]]
del clean_txts[idxs[-3]]

In [99]:
'' in clean_txts

False

In [100]:
del sample_audios[idxs[-1]]
del sample_audios[idxs[-2]]
del sample_audios[idxs[-3]]

## Tokenizer

In [101]:
def character_dict():
    alphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
    supported = alphabet.split()

    char_map = {}
    char_map[""] = 0
    char_map["<SPACE>"] = 1
    idx = 2
    for c in supported:
        char_map[c] = idx
        idx += 1
    index_map = {v: k for k, v in char_map.items()}
    return char_map, index_map

In [102]:
char_map, index_map = character_dict()

In [103]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [104]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        elif c in alphabets:
            ch = char_map[c]
        else:
            print(c)
            print('character not found')
            break
        int_sequence.append(ch)
    return np.array(int_sequence)

In [105]:
def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    textch = []
    for c in int_sequence:
        ch = index_map[c]
        textch.append(ch)
    text = ''.join(textch)
    text = text.replace('<SPACE>', ' ')
    return text

In [106]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, audios, texts, batch_size=32):
        self.audios = audios
        self.texts = texts
        self.batch_size = batch_size
        self.steps = int(len(self.audios) // self.batch_size)
        self.on_epoch_end()

    def __len__(self):
        return self.steps

    def on_epoch_end(self):
        self.indexes = np.arange(self.steps*self.batch_size)
        # np.random.shuffle(self.indexes)

    def __getitem__(self, index):
        indexes = self.indexes[int(index*self.batch_size):int((index+1)*self.batch_size)]
    
        batch_audios = [self.audios[int(i)] for i in indexes]
        batch_texts = [self.texts[int(i)] for i in indexes]
        
        return  self.data_generation(batch_audios, batch_texts)
    
    def data_generation(self, batch_audios, batch_texts):

        longest_audio = max([len(i) for i in batch_audios])
        longest_txt = max([len(i) for i in batch_texts])

        audios          = np.zeros([int(self.batch_size), longest_audio], dtype="float32")
        txts            = np.zeros([int(self.batch_size), longest_txt], dtype="int64")
        audio_length    = np.zeros([int(self.batch_size)], dtype="int64")
        txt_length      = np.zeros([int(self.batch_size)], dtype="int64")

        i = 0
        for audio, txt in zip(batch_audios, batch_texts):

            txt_len = len(txt)

            txt = text_to_int_sequence(txt)
       
            txts[i,: txt_len] = txt

            audio_len = len(audio)

            audios[i, :audio_len] = audio

            audio_length[i] = audio_len
            txt_length[i] = txt_len

            i+=1          
            
        outputs = {'ctc': np.zeros([self.batch_size])}
        inputs = {
                    'the_input':    tf.convert_to_tensor(audios), 
                    'the_labels':   tf.convert_to_tensor(txts), 
                    'input_length': tf.convert_to_tensor(audio_length), 
                    'label_length': tf.convert_to_tensor(txt_length)
                }
        return (inputs, outputs)


In [116]:
dg = DataGenerator(sample_audios, clean_txts)

In [117]:
len(dg)

34

In [118]:
dg[5]

({'input_length': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([ 76292,  49616,  98786,  64826,  52920, 113343,  54906,  50274,
          61960,  88644, 128772,  78278,  84671,  69458,  48289,  54686,
          58433,  48510,  69017,  50054,  77175,  78938,  60858,  91508,
         104741,  90189,  69458, 100769,  69899,  62180,  63284, 106942])>,
  'label_length': <tf.Tensor: shape=(32,), dtype=int64, numpy=
  array([50, 15, 97, 49, 49,  5, 56, 20, 51, 74, 60, 46, 62, 52, 44, 33, 41,
         39, 58, 48, 53, 62, 74, 71, 57, 77, 27, 82, 18, 54, 35, 76])>,
  'the_input': <tf.Tensor: shape=(32, 128772), dtype=float32, numpy=
  array([[-0.00383717, -0.00409213, -0.0034892 , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.03015642, -0.02410989, -0.03892324, ...,  0.        ,
           0.        ,  0.        ],
         [-0.02567921, -0.02798967, -0.02170274, ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [-0.00794118, -0.01024136,

In [119]:
batch1 = dg[0][0]


In [120]:
batch1['the_labels']

<tf.Tensor: shape=(32, 109), dtype=int64, numpy=
array([[11,  2, 15, ...,  0,  0,  0],
       [ 9,  2, 13, ...,  0,  0,  0],
       [14,  2, 14, ...,  0,  0,  0],
       ...,
       [12, 24,  2, ...,  0,  0,  0],
       [12,  2, 21, ...,  0,  0,  0],
       [15, 10,  1, ...,  0,  0,  0]])>

In [121]:
batch1['the_input']

<tf.Tensor: shape=(32, 117746), dtype=float32, numpy=
array([[-1.9447643e-02, -2.3805585e-02, -1.9458510e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 6.4895018e-03, -9.7726457e-05, -6.6772741e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.2900349e-02, -1.4687851e-02, -1.2652838e-02, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [ 3.2423830e-03,  2.7590861e-05, -3.5644281e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 8.2196379e-03,  9.0677310e-03,  7.8771934e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.0445359e-04, -1.1000659e-03, -1.9130670e-03, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)>

### ctc


In [113]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [114]:
def input_lengths_lambda_func(args):
    input_length = args
    return tf.cast(tf.math.floor(input_length/hop_size)-1, dtype="float32")

In [115]:
def add_ctc_loss(model_builder):
    the_labels      = Input(name='the_labels',      shape=(None,), dtype='float32')
    input_lengths   = Input(name='input_length',    shape=(1,), dtype='float32')
    label_lengths   = Input(name='label_length',    shape=(1,), dtype='float32')

    input_lengths2 = Lambda(input_lengths_lambda_func)(input_lengths)
    if model_builder.output_length:
         output_lengths  = Lambda(model_builder.output_length)(input_lengths2)
    else:
         output_lengths  = input_lengths2
    
    # CTC loss is implemented in a lambda layer
    loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([model_builder.output, the_labels, output_lengths, label_lengths])
    model = Model( inputs=[model_builder.input, the_labels, input_lengths, label_lengths],  outputs=loss_out)
    return model

## LogMelSpectrogram

In [122]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.
        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.
        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config

## Models

In [123]:
def preprocessin_model(sample_rate, fft_size, frame_step, n_mels, mfcc=False):

    input_data = Input(name='input', shape=(None,), dtype="float32")
    featLayer = LogMelSpectrogram(
        fft_size=fft_size,
        hop_size=frame_step,
        n_mels=n_mels,
        
        sample_rate=sample_rate,
        f_min=0.0,
        
        f_max=int(sample_rate / 2)
    )(input_data)
    
    x = BatchNormalization()(featLayer)
    model = Model(inputs=input_data, outputs=x, name="preprocessin_model")

    return model

In [124]:
def simple_rnn_model(input_dim, output_dim=224):

    input_data = Input(name='the_input', shape=(None, input_dim))
    simp_rnn = GRU(output_dim, return_sequences=True,
                   implementation=2, name='rnn')(input_data)
    y_pred = Activation('softmax', name='softmax')(simp_rnn)
    model = Model(inputs=input_data, outputs=y_pred, name="simple_rnn_model")
    model.output_length = lambda x: x
    return model


In [125]:
def BidirectionalRNN(input_dim, rnn_layers=2, units=400, drop_out=0.5, act='tanh', output_dim=224):

    input_data = Input(name='the_input', shape=(
        None, input_dim))

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(input_data)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    for i in range(rnn_layers - 2):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    x = Bidirectional(LSTM(units,  activation=act,
                      return_sequences=True, implementation=2))(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out)(x)

    time_dense = TimeDistributed(Dense(output_dim))(x)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="BidirectionalRNN")

    return model

In [126]:
def conv_rnn(n_mels, output_dim=224, rnn_layers=4, units=400, drop_out=0.5, act='tanh'):

    input_data = Input(name='the_input', shape=(None, n_mels, 1))

    y = Conv2D(32, (3, 3), padding='same')(input_data)  # was 32
    y = Activation('relu')(y)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(y)

    x = Conv2D(64, (3, 3), padding='same')(y)  # was 32
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Conv2D(128, (3, 3), padding='same')(x)
    x = Activation('relu')(x)
    y = BatchNormalization()(y)

    x = MaxPooling2D((1, 2))(x)

    x = Dense(128)(x)
    x = Dense(64)(x)
    x = Dense(32)(x)

    x = Reshape((-1, x.shape[-1] * x.shape[-2]))(x)

    for i in range(rnn_layers):
        x = Bidirectional(
            LSTM(units, activation=act, return_sequences=True))(x)
        x = BatchNormalization()(x)
        x = Dropout(drop_out)(x)

    bn_rnn = BatchNormalization()(x)

    time_dense = TimeDistributed(Dense(output_dim))(bn_rnn)

    y_pred = Activation('softmax', name='softmax')(time_dense)

    model = Model(inputs=input_data, outputs=y_pred, name="custom_model")

    return model

In [127]:
def train(model_builder, 
          data_gen,
          epochs, 
          verbose=1,
          optimizer=SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True, clipnorm=5),
          ):    
              
    model = add_ctc_loss(model_builder)

    model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=optimizer)
    print(model.summary())

    # add checkpointer
    checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5', verbose=1)
    early_stopping = EarlyStopping( monitor="val_loss", patience=10, restore_best_weights=True)

    hist = model.fit_generator(generator=data_gen,
                               epochs=epochs,
                               callbacks=[checkpointer, early_stopping], 
                               verbose=verbose, 
                               use_multiprocessing=False)
    
    # save model loss
    with open('/content/drive/MyDrive/structured_data/train/models/cnn_rnn.pickle', 'wb') as f:
        pickle.dump(hist.history, f)

### Trainig

In [128]:

sample_rate = 22050
fft_size = 1024
frame_step = 512
n_mels = 128

batch_size = 32
epochs = 100
data_len = len(clean_txts)
output_dim = len(char_map) + 2


In [129]:
dg = DataGenerator(sample_audios, clean_txts, batch_size)

In [130]:
preprocess_model = preprocessin_model(sample_rate, fft_size, frame_step, n_mels)
preprocess_model.summary()


Model: "preprocessin_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
log_mel_spectrogram_3 (LogMe (None, None, 128, 1)      0         
_________________________________________________________________
batch_normalization_27 (Batc (None, None, 128, 1)      4         
Total params: 4
Trainable params: 2
Non-trainable params: 2
_________________________________________________________________


In [131]:
speech_model = conv_rnn(n_mels, output_dim = output_dim)
speech_model.summary()


Model: "custom_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None, 128, 1)]    0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, None, 128, 32)     320       
_________________________________________________________________
activation_9 (Activation)    (None, None, 128, 32)     0         
_________________________________________________________________
batch_normalization_28 (Batc (None, None, 128, 32)     128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, None, 128, 64)     18496     
_________________________________________________________________
activation_10 (Activation)   (None, None, 128, 64)     0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, None, 64, 64)     

In [132]:
def build_model(output_dim, custom_model, preprocess_model, calc=None):

    input_audios = Input(name='the_input', shape=(None,))
    pre = preprocess_model(input_audios)
    pre = tf.squeeze(pre, [3])

    y_pred = custom_model(pre)
    model = Model(inputs=input_audios, outputs=y_pred, name="model_builder")
    model.output_length = calc

    return model

In [133]:
model = build_model(output_dim, speech_model, preprocess_model)
model.summary()


Model: "model_builder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       [(None, None)]            0         
_________________________________________________________________
preprocessin_model (Function (None, None, 128, 1)      4         
_________________________________________________________________
tf.compat.v1.squeeze_3 (TFOp (None, None, 128)         0         
_________________________________________________________________
custom_model (Functional)    (None, None, 30)          16249278  
Total params: 16,249,282
Trainable params: 16,241,216
Non-trainable params: 8,066
_________________________________________________________________


In [134]:
model.load_weights('/content/drive/MyDrive/structured_data/train/models/cnn_rnn3.h5')

In [ ]:
# !pip install mlflow --quiet

In [ ]:
# import mlflow

In [139]:
# mlflow.set_experiment('Speech Model-RNN-baseline')
# mlflow.tensorflow.autolog()
hop_size = 512
train(model, dg, epochs=100)


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
preprocessin_model (Functional) (None, None, 128, 1) 4           the_input[0][0]                  
__________________________________________________________________________________________________
tf.compat.v1.squeeze_3 (TFOpLam (None, None, 128)    0           preprocessin_model[0][0]         
__________________________________________________________________________________________________
input_length (InputLayer)       [(None, 1)]          0                                            
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


34/34 [==============================] - 27s 567ms/step - loss: 48.5397

Epoch 00001: saving model to /content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5
Epoch 2/100
34/34 [==============================] - 20s 582ms/step - loss: 46.7798

Epoch 00002: saving model to /content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5
Epoch 3/100
34/34 [==============================] - 20s 580ms/step - loss: 45.7685

Epoch 00003: saving model to /content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5
Epoch 4/100
34/34 [==============================] - 20s 571ms/step - loss: 44.1256

Epoch 00004: saving model to /content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5
Epoch 5/100
34/34 [==============================] - 19s 565ms/step - loss: 42.7439

Epoch 00005: saving model to /content/drive/MyDrive/structured_data/train/models/cnn_rnn.h5
Epoch 6/100
34/34 [==============================] - 20s 573ms/step - loss: 41.3259

Epoch 00006: saving model to /content/driv

In [ ]:
!pip install jiwer --quiet

     |████████████████████████████████| 50 kB 4.2 MB/s 


In [ ]:
from jiwer import wer

In [47]:
def predict(data_gen,  num_elem=1, index=0):
    
    pred_data = data_gen.__getitem__(index)

    pred_audios = pred_data[0]["the_input"]
    pred_labels = pred_data[0]["the_labels"]
    
    y_pred = model.predict(pred_audios)

    input_shape = tf.keras.backend.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * tf.keras.backend.cast(input_shape[1], 'float32')
    prediction = tf.keras.backend.ctc_decode(y_pred, input_length, greedy=False)[0][0]
    

    for i in range(0, num_elem):  # only on clean data
        
        pred = K.eval(prediction[i]).flatten().tolist()
        pred = [i for i in pred if i != -1]



        ground_truth = int_sequence_to_text(pred_labels[i].numpy())
        hypothesis   = ''.join(int_sequence_to_text(pred))
        # error        = wer(ground_truth, hypothesis)
                
        print('-'*48 + ' ' + str(i) + ' ' + '-'*48)
        print('True transcription:\n' + '\n' + ground_truth)
        print('-'*100)
        print('Predicted transcription:\n' + '\n' + hypothesis)
        # print('-'*100)
        # print('Word Error Rate:' + str(error))
        print('\n')

In [48]:
rows1 = []
parent_dir1 = "./SWH-05-20101107"
files1 = os.listdir(parent_dir1)
for f in files1:
    audio, fs = librosa.load(f"{parent_dir1}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows1.append(row)
rows1[:5]

[{'audio': array([0.0043117 , 0.00573289, 0.00458423, ..., 0.01505196, 0.02441092,
         0.01784338], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part1'},
 {'audio': array([ 0.00038796,  0.0003749 ,  0.00038821, ..., -0.13299914,
         -0.15665275, -0.09959406], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part10'},
 {'audio': array([-0.00805496, -0.00836756, -0.00562658, ...,  0.00168179,
         -0.0017211 ,  0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part100'},
 {'audio': array([0.01148875, 0.01478087, 0.01499198, ..., 0.07277071, 0.03534885,
         0.        ], dtype=float32),
  'filename': 'SWH-05-20101107_16k-emission_swahili_05h30_-_06h00_tu_20101107_part101'},
 {'audio': array([-0.0029805 , -0.00687017, -0.00985629, ...,  0.0279939 ,
          0.0324325 ,  0.        ], dtype=float32),
  'filename': 'SWH-05-20

In [49]:
sample_audios1 = []
for row in rows1:
    audio = row['audio']
    sample_audios1.append(audio)
sample_audios1[:5]

[array([0.0043117 , 0.00573289, 0.00458423, ..., 0.01505196, 0.02441092,
        0.01784338], dtype=float32),
 array([ 0.00038796,  0.0003749 ,  0.00038821, ..., -0.13299914,
        -0.15665275, -0.09959406], dtype=float32),
 array([-0.00805496, -0.00836756, -0.00562658, ...,  0.00168179,
        -0.0017211 ,  0.        ], dtype=float32),
 array([0.01148875, 0.01478087, 0.01499198, ..., 0.07277071, 0.03534885,
        0.        ], dtype=float32),
 array([-0.0029805 , -0.00687017, -0.00985629, ...,  0.0279939 ,
         0.0324325 ,  0.        ], dtype=float32)]

In [50]:
meta_df = pd.read_csv('/content/drive/MyDrive/metadata.csv')

In [51]:
txts1 = []
for row in rows1:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts1.append(txt)

txts1[:5]

[array([['karibu katika matangazo yetu ya asubuhi ya idhaa ya kiswahili']],
       dtype=object),
 array([['kwenda na kuondoka jarkata indonesia']], dtype=object),
 array([['lazima uwe na outside support mataifa ya nje yakuunge mkono']],
       dtype=object),
 array([['nne lazima uwe na military support eeh']], dtype=object),
 array([['lazima akuwe mshindi tu hapo unaona je kwa nini wasiwe washindi']],
       dtype=object)]

In [52]:
txts1 = np.array(txts1).reshape(-1)

In [53]:
txts1[:5]

array(['karibu katika matangazo yetu ya asubuhi ya idhaa ya kiswahili',
       'kwenda na kuondoka jarkata indonesia',
       'lazima uwe na outside support mataifa ya nje yakuunge mkono',
       'nne lazima uwe na military support eeh',
       'lazima akuwe mshindi tu hapo unaona je kwa nini wasiwe washindi'],
      dtype=object)

In [54]:
clean_txts1 = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts1:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts1.append(clean_txt)

In [55]:
clean_txts1[:5]

['karibu katika matangazo yetu ya asubuhi ya idhaa ya kiswahili',
 'kwenda na kuondoka jarkata indonesia',
 'lazima uwe na outside support mataifa ya nje yakuunge mkono',
 'nne lazima uwe na military support eeh',
 'lazima akuwe mshindi tu hapo unaona je kwa nini wasiwe washindi']

In [56]:
'' in clean_txts1

False

In [57]:
dg1 = DataGenerator(sample_audios1[:5], clean_txts1[:5], 5)

In [140]:
predict(dg1, 4)

------------------------------------------------ 0 ------------------------------------------------
True transcription:

karibu katika matangazo yetu ya asubuhi ya idhaa ya kiswahili
----------------------------------------------------------------------------------------------------
Predicted transcription:

karibu kaia natanazo yeti ya subuia idha ya kiswahili 


------------------------------------------------ 1 ------------------------------------------------
True transcription:

kwenda na kuondoka jarkata indonesia
----------------------------------------------------------------------------------------------------
Predicted transcription:

kuwenda kuna kundoka jijya kata indoneziaoanfuyuhyhya b


------------------------------------------------ 2 ------------------------------------------------
True transcription:

lazima uwe na outside support mataifa ya nje yakuunge mkono
----------------------------------------------------------------------------------------------------
Predicte